## Statistical Learning and Deep Learning HW5

### Q1

In [30]:
import numpy as np
import pandas as pd
import glob
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models

In [28]:
datasets = ['train', 'valid' ,'test']
labels = ['blazer', 'cardigan', 'coat', 'jacket']
base_path = './photos'

In [34]:
df = pd.DataFrame(columns=labels, index=datasets)
for ds in datasets:
    for lb in labels:
        basepath = os.path.join(f'{base_path}/{ds}/{lb}/', '*.jpg')
        cand_fn = glob.glob(basepath)
        df[lb][ds] = len(cand_fn)
df['total'] = df.sum(axis=1).astype('int')
print(df)

      blazer cardigan coat jacket  total
train     97      237  296    411   1041
valid      7       36   27     35    105
test       9       42   43     52    146


In [35]:
print('Ratio:')
df = df.drop(['total'], axis=1)
print (df.div(df.sum(axis=1), axis=0))

Ratio:
         blazer  cardigan      coat    jacket
train  0.093180  0.227666  0.284342  0.394813
valid  0.066667  0.342857  0.257143  0.333333
test   0.061644  0.287671  0.294521  0.356164


Given the number of instances of each image type, I suggest that the accuracy of the classification task will be jacket > coat > cardigan > blazer. This follows the hypothesis that larger number of instances in training set causes higher classification accuracy.

### Q2

In [5]:
# Image transformations
image_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.RandomResizedCrop(size=(224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=20),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor()
    ]),
    
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor()
    ]),
}

In [6]:
# Datasets from folders
data = {
    'train':
        ImageFolder(root=f'{base_path}/train/', transform=image_transforms['train']),
    'valid':
        ImageFolder(root=f'{base_path}/valid/', transform=image_transforms['valid']),
    'test':
        ImageFolder(root=f'{base_path}/test/', transform=image_transforms['test'])
}

In [7]:
# Dataloader
batch_size = 32
dataloaders = {
    'train':
        DataLoader(data['train'], batch_size=batch_size, shuffle=True, num_workers=10),
    'valid':
        DataLoader(data['valid'], batch_size=batch_size, shuffle=True, num_workers=10),
    'test':
        DataLoader(data['test'], batch_size=batch_size, shuffle=True, num_workers=10)
}

In [8]:
# device
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')
print(torch.cuda.current_device())

using cuda
0


In [20]:
# load pretrained resnet50 and set the output dimension to 4
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 4)
model.to(device)
print(f'model at {device}')

model at cuda


In [21]:
# loss
loss_fn = nn.CrossEntropyLoss()

In [22]:
def cross_entropy_loss(model, data_loader):
    loss = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss += loss_fn(outputs, target)
    return loss

Train with SGD.

In [23]:
def train(optim, model, weight_path, fix_weight=True, early_stop_patient=20, max_epoch=200):
    best_valid_loss = np.inf
    best_valid_epoch = 0
    for epoch in range(max_epoch):
        # train
        for batch, (data, target) in enumerate(dataloaders['train']):
            data, target = data.to(device), target.to(device)
            model.train()
            optim.zero_grad()
            outputs = model(data)
            loss = loss_fn(outputs, target)
            loss.backward()
            optim.step()
            
        # validation
        model.eval()
        valid_loss = cross_entropy_loss(model, dataloaders['valid']).cpu().numpy()
 
        # update weight if lower validation loss
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_valid_epoch = epoch
            torch.save(model, weight_path)
        # early stopping
        elif (epoch - best_valid_epoch >= early_stop_patient):
            print(f'early stopping at epoch {epoch}')
            return
        
        print(f'epoch {epoch}: train_loss = {loss.detach().cpu().numpy():.3f}, valid_loss = {valid_loss:.3f}, best_valid = {best_valid_loss:.3f}')
        

In [24]:
optim = torch.optim.SGD(model.parameters(), lr=0.1)
train(optim, model, './Q2_weight')

epoch 0: train_loss = 2.649, valid_loss = 48.804, best_valid = 48.804
epoch 1: train_loss = 8.977, valid_loss = 25.344, best_valid = 25.344
epoch 2: train_loss = 14.014, valid_loss = 48.412, best_valid = 25.344


KeyboardInterrupt: 

In [26]:
saved_model = torch.load('./Q2_weight')
with torch.no_grad():
    for batch, (data, target) in enumerate(dataloaders['test']):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, preds = torch.max(outputs, 1)
        print(target)
        print(preds)

tensor([2, 2, 3, 3, 1, 3, 2, 3, 3, 3, 3, 2, 1, 3, 2, 3, 0, 1, 0, 3, 2, 2, 1, 2,
        1, 1, 3, 2, 2, 2, 1, 3], device='cuda:0')
tensor([3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1,
        1, 1, 3, 1, 1, 1, 1, 1], device='cuda:0')
tensor([3, 1, 1, 1, 3, 1, 2, 2, 3, 3, 2, 3, 1, 3, 3, 1, 3, 1, 3, 0, 2, 1, 1, 2,
        1, 1, 2, 1, 3, 2, 2, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([3, 0, 1, 2, 1, 1, 1, 3, 2, 1, 1, 2, 3, 2, 3, 3, 2, 2, 1, 3, 1, 1, 2, 3,
        1, 3, 0, 1, 3, 3, 3, 2], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
        1, 3, 1, 1, 3, 1, 1, 1], device='cuda:0')
tensor([2, 3, 3, 2, 2, 3, 3, 1, 0, 1, 3, 1, 2, 3, 3, 0, 2, 1, 2, 2, 2, 3, 3, 3,
        3, 2, 3, 2, 1, 2, 2, 1], device='cuda:0')
tensor([1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1,
        3,

In [ ]:
# all_lr = [0.001, 0.01, 0.1, 1, 5]
# for lr in all_lr:
#     optim = torch.optim.SGD(model.parameters(), lr=lr)
    